In [1]:
import numpy as np
import csv
import pandas
import random
import cv2
from pygame import mixer
from keras.datasets import imdb
from keras.models import Sequential, load_model
from keras.layers import Dense , Dropout
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import collections
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


Using TensorFlow backend.


In [2]:
data_path='C:/Users/RAHUL/projects/Many_to_one'

In [3]:
filename='C:/Users/RAHUL/Downloads/compiled.csv'

In [4]:
def read_csv(filename):
    dataframe = pandas.read_csv(filename, header=None)
    #ds = dataframe.sample(frac=1)
    dataset = dataframe.values
    X = dataset[:,0]
    #Y = dataset[:,1]
    #ds.to_csv('compiled_predicted.csv')
    return X
X=read_csv(filename)

In [5]:
def generate_all_sentences(X):
    sentences=[]
    for row in X:
        #print(row)
        sentences.append(tf.compat.as_str(row).split())
    return sentences
sentences=generate_all_sentences(X)

In [6]:
def get_word_list(sentences):
    word_list=[]
    for sentence in sentences:
        for word in sentence:
            word_list.append(word)
    return word_list
word_list=get_word_list(sentences)

In [7]:
def file_to_word_ids(word_list,sentences,word_to_id):
    new_sentences=[]
    for sentence in sentences:
        new_sentences.append([word_to_id[word] for word in sentence if word in word_to_id])
    #data = word_list
    return new_sentences

def get_dict(word_list,sentences):
    counter = collections.Counter(word_list)
    count_pairs = sorted(counter.items(), key=lambda x: (-x[1], x[0]))

    words, _ = list(zip(*count_pairs))
    word_to_id = dict(zip(words, range(len(words))))
    vocabulary = len(word_to_id)
    reverse_dictionary = dict(zip(word_to_id.values(), word_to_id.keys()))
    train_data = file_to_word_ids(word_list,sentences, word_to_id)
    return word_to_id,reverse_dictionary,vocabulary, train_data
dictionary,reverse_dictionary,vocabulary, X_train=get_dict(word_list,sentences)
print(len(X_train))
print(vocabulary)

230
782


In [8]:
max_review_length = 18
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)

In [10]:
def predict_quote_classes():
    #should see how to test model in keras
    #I can already train all 3 models and then load all 3 models
    filename="binary_crossentropy/single 32 lstm/model-10-0.5622.hdf5"
    model = load_model(data_path + "/" + filename)
    xnew=X_train
    ynew = model.predict_classes(xnew)
    #happy=list(range(1:117))
    #motivation=list(range(117:231))
    i=0
    happy=[]
    motivation=[]
    #check whether index should start from 0 or 1
    for clas in ynew:
        if(clas==1):
            #means the quote is happy
            happy.append(i)
        else:
            motivation.append(i)
        i=i+1
    return happy,motivation

def map_quote(happy,motivation,input_emotion):
    if (input_emotion=="happy"):
        #maps to happy quote
        #print("Displaying happy quote")
        reply_quote=random.randint(0,len(happy))
        #between 0 and len(happy) -1
        #print (reply_quote+1)
        reply_quote=happy[reply_quote]
        #print(reply_quote)
    elif (input_emotion=="sad"):
        #maps to motivated quote
        #print("Displaying motivated quote")
        reply_quote=random.randint(0,len(motivation))
        #print (reply_quote+1)
        reply_quote=motivation[reply_quote]
        #print(reply_quote)
    elif (input_emotion=="disgust"):
        #maps to motivated quote
        #print("Displaying happy quote")
        reply_quote=random.randint(0,len(happy))
        #print (reply_quote+1)
        reply_quote=happy[reply_quote]
        #print(reply_quote)
    elif (input_emotion=="fear"):
        #maps to motivated quote
        #print("Displaying motivated quote")
        reply_quote=random.randint(0,len(motivation))
        #print (reply_quote+1)
        reply_quote=motivation[reply_quote]
        #print(reply_quote)
    elif (input_emotion=="anger"):
        #maps to motivated quote
        #print("Displaying motivated quote")
        reply_quote=random.randint(0,len(motivation))
        #print (reply_quote+1)
        reply_quote=motivation[reply_quote]
        #print(reply_quote)
    elif (input_emotion=="surprise"):
        #maps to motivated quote
        #print("Displaying happy quote")
        reply_quote=random.randint(0,len(happy))
        #print (reply_quote+1)
        reply_quote=happy[reply_quote]
        #print(reply_quote)
    return reply_quote
def display_quote(reply_quote_index):
    #filename_pred='C:/Users/RAHUL/Downloads/compiled_predicted.csv'
    filename='C:/Users/RAHUL/Downloads/compiled.csv'
    dataframe = pandas.read_csv(filename, header=None)
    dataset = dataframe.values
    X = dataset[:,0]
    return(X[reply_quote_index])
def predict_image_classes():
    #first load the keras image classification model
    #next predict image
    #return class
    return "happy"
def play_song(reply_music_index=0):
    mixer.init()
    mixer.music.load('C:/Users/RAHUL/Downloads/taki_taki.mp3')
    mixer.music.play(start=15.0)
def live_feed():
    cv2.namedWindow("EASY")
    vc = cv2.VideoCapture(0)

    if vc.isOpened(): # try to get the first frame
        rval, frame = vc.read()
    else:
        rval = False

    while rval:
        cv2.imshow("EASY", frame)
        rval, frame = vc.read()
        key = cv2.waitKey(20)
        if key == 27: # exit on ESC
            break

    cv2.destroyWindow("EASY")
    vc.release()

In [11]:
#Step 0 - 
#set up database lists
#WARNING : RUN ONCE AT THE BEGGNING OF THE APP
happy, motivation=predict_quote_classes()
#happy_music, motivation_music=predict_music_classes()

In [12]:
#while(1):

#step 1-
#getting the face classification model predictions -
#live_feed()
input_emotion=predict_image_classes()

#step 2-
#getting the reply quote index from the model
reply_quote_index=map_quote(happy,motivation,input_emotion)
#reply_music_index=map_quote(happy_music,motivation_music,input_emotion)

#step 3-
#now display the quote from the CSV file
#actually make it compiled_pred, else the model has to store the happy, motivated list in the memory 
print(display_quote(reply_quote_index))
#play_song(reply_music_index)
play_song()
#now repeat this after every song gets played and quit when the user types quit

true happiness arises in the first place from the enjoyment of ones self


In [ ]:
#have to make music classifier and then integrate images with this

In [13]:
mixer.quit()